In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# Define the file path
file_path = '/mnt/ceph/users/lbrown/MouseData/Rebecca/230212_stack6/LineageGraph.json'

# Load the JSON data
with open(file_path, 'r') as f:
    data = json.load(f)

In [5]:
nodes_data = data["Nodes"]
edges_data = data["Edges"]

# Create a dictionary for node properties
nodes = {
    node["Name"]: {"frame": int(node["Name"].split("_")[0])} for node in nodes_data
}

# Build parent-child relationships
parent_of = {edge["EndNodes"][1]: edge["EndNodes"][0] for edge in edges_data}
children_of = {}
for edge in edges_data:
    parent_name = edge["EndNodes"][0]
    child_name = edge["EndNodes"][1]
    if parent_name not in children_of:
        children_of[parent_name] = []
    children_of[parent_name].append(child_name)

# Identify roots (nodes with no parents)
root_nodes = sorted([name for name in nodes if name not in parent_of])

pos = {}
x_offset = 0


def layout_tree(node_name, current_x):
    """Recursively assign x positions to nodes."""
    global x_offset

    if node_name in pos:
        return

    pos[node_name] = (current_x, nodes[node_name]["frame"])

    children = children_of.get(node_name, [])

    if not children:
        x_offset += 1
        return

    start_x = current_x - (len(children) - 1) / 2.0

    child_positions = []
    for i, child in enumerate(sorted(children)):
        temp_x = start_x + i
        is_occupied = any(
            p[1] == nodes[child]["frame"] and abs(p[0] - temp_x) < 0.5
            for p in pos.values()
        )
        if is_occupied:
            temp_x = x_offset + 1

        layout_tree(child, temp_x)
        child_positions.append(pos[child][0])

    if child_positions:
        pos[node_name] = (np.mean(child_positions), nodes[node_name]["frame"])


# Layout all trees
for root in root_nodes:
    if root not in pos:
        layout_tree(root, x_offset)


plt.figure(figsize=(30, 22))

# Plot edges
for child, parent in parent_of.items():
    if parent in pos and child in pos:
        plt.plot(
            [pos[parent][0], pos[child][0]],
            [pos[parent][1], pos[child][1]],
            "k-",
            lw=0.7,
            alpha=0.6,
        )

# Plot nodes and labels
for name, p in pos.items():
    color = "lightblue"
    if name in children_of:
        if len(children_of[name]) > 1:
            color = "green"
    else:
        color = "red"

    plt.plot(
        p[0],
        p[1],
        "o",
        markersize=7,
        color=color,
        markeredgecolor="black",
        markeredgewidth=0.5,
    )
    plt.text(p[0] + 0.1, p[1], name, fontsize=5, ha="left", va="center")

plt.xlabel("Individual Cell Lineages", fontsize=16)
plt.ylabel("Frame Number", fontsize=16)
plt.title("Cell Lineage Tree with Node Labels", fontsize=20)
plt.gca().invert_yaxis()
plt.xticks([])
plt.grid(axis="y", linestyle="--", linewidth=0.5)

# Create a legend
from matplotlib.lines import Line2D

legend_elements = [
    Line2D(
        [0],
        [0],
        marker="o",
        color="w",
        label="Stasis (Continues)",
        markerfacecolor="lightblue",
        markersize=12,
    ),
    Line2D(
        [0],
        [0],
        marker="o",
        color="w",
        label="Mitosis (Division)",
        markerfacecolor="green",
        markersize=12,
    ),
    Line2D(
        [0],
        [0],
        marker="o",
        color="w",
        label="Death / End of Track",
        markerfacecolor="red",
        markersize=12,
    ),
]

plt.legend(handles=legend_elements, loc="upper right", fontsize=14)
plt.tight_layout()
# plt.savefig("lineage_tree_with_labels.svg", dpi=800)
plt.close()

print("Lineage tree with node labels saved as lineage_tree_with_labels.png")

Lineage tree with node labels saved as lineage_tree_with_labels.png


In [9]:
# Get the list of children created by mitotic events (parents with exactly two children)
children_frames = [
    (child, nodes[child]["frame"])
    for parent, children in children_of.items()
    if len(children) == 2
    for child in children
]

# Print the list
print("Currently created children with their frames:")
print(children_frames)

Currently created children with their frames:
[('025_011', 25), ('025_014', 25), ('026_015', 26), ('026_016', 26), ('028_003', 28), ('028_010', 28), ('064_015', 64), ('064_018', 64), ('066_017', 66), ('066_019', 66), ('066_016', 66), ('066_018', 66), ('067_014', 67), ('067_021', 67), ('067_018', 67), ('067_023', 67), ('067_022', 67), ('067_024', 67), ('067_013', 67), ('067_020', 67), ('068_015', 68), ('068_027', 68), ('069_020', 69), ('069_026', 69), ('070_014', 70), ('070_019', 70), ('070_008', 70), ('070_021', 70), ('071_022', 71), ('071_024', 71), ('072_016', 72), ('072_020', 72), ('072_021', 72), ('072_024', 72), ('075_017', 75), ('075_028', 75), ('077_006', 77), ('077_033', 77), ('117_018', 117), ('117_020', 117), ('118_031', 118), ('118_033', 118), ('120_024', 120), ('120_030', 120), ('122_029', 122), ('122_032', 122), ('122_028', 122), ('122_038', 122), ('125_025', 125), ('125_036', 125), ('126_030', 126), ('126_038', 126), ('126_039', 126), ('126_040', 126), ('127_034', 127), (